# Plant Species Classifier
Welcome to the Plant Species Classifer application! To get started, run the following block of code to ensure that Google Colab is using the latest versions of TensorFlow and Keras. This is required to import the trained model.

In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade keras

Next, we need to import the required libraries, download files for the trained model and visualization support, and create a few utility functions that will be used in the interactive application. Run the following code block to accomplish this.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
import gdown

# Public image URLs from Google Drive.
model_url = 'https://drive.google.com/uc?id=1rnCxsFkkO91rPBTsQFBJ4_vglk8uFGVM'
training_history_url = 'https://drive.google.com/uc?id=1XBe_i1Wm1muATKwFn0-yefOmLUxgu1TF'
test_images_url = 'https://drive.google.com/uc?id=1BBvYM11ruTZIp1me83WfHoEZBb98V2Mz'

# Paths to store the downloaded images.
model_path = 'plant_species_classifier.keras'
training_history_path = 'training_history.json'
test_images_path = 'test_images.zip'

# Download the trained model, training history in JSON format, and test images for the confusion matrix.
gdown.download(model_url, model_path, quiet=False)
gdown.download(training_history_url, training_history_path, quiet=False)
gdown.download(test_images_url, test_images_path, quiet=False)

model = load_model(model_path)

# List of plant species supported by the model.
plant_species = [
    "aloevera",
    "banana",
    "bilimbi",
    "cantaloupe",
    "cassava",
    "coconut",
    "corn",
    "cucumber",
    "curcuma",
    "eggplant",
    "galangal",
    "ginger",
    "guava",
    "kale",
    "longbeans",
    "mango",
    "melon",
    "orange",
    "paddy",
    "papaya",
    "peperchili",
    "pineapple",
    "pomelo",
    "shallot",
    "soybeans",
    "spinach",
    "sweetpotatoes",
    "tobacco",
    "waterapple",
    "watermelon",
]

# Preprocess the image in preparation for feeding it to the model.
def preprocess_image(img):
    img = img.convert("RGB")  # Convert the image to RGB format.
    img = img.resize((224, 224))  # Resize the image to the input size of the model.
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  # Normalize the image.
    return img

# Preprocess the image and return the prediction from the model.
def predict_image(img):
    preprocessed_img = preprocess_image(img)
    predictions = model.predict(preprocessed_img)
    return predictions

def plot_prediction_confidence(predictions, class_labels):
    plt.figure(figsize=(10, 5))
    plt.barh(class_labels, predictions[0], color='skyblue')
    plt.xlabel('Confidence')
    plt.ylabel('Class')
    plt.title('Prediction Confidence for User-Uploaded Image')
    plt.show()

The following code block will prompt you to upload an image, and will return the predicted plant species. This will also display a confidence for the prediction as a bar graph.

In [ ]:
from google.colab import files
uploaded = files.upload()

# Make a prediction for the file that the user uploaded, and display the uploaded image.
for file_name in uploaded.keys():
    img = Image.open(BytesIO(uploaded[file_name]))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

    # Get a prediction of the plant species from the model.
    predictions = predict_image(img)
    predicted_class = np.argmax(predictions, axis=1)

    # Display the prediction and the confidence level for the prediction.
    print(f'Predicted class: {plant_species[predicted_class[0]]}')
    plot_prediction_confidence(predictions, plant_species)

# Dashboards

The following two code blocks display two separate visualizations: one for the training and validation accuracy and loss, and one for a confusion matrix of the testing data in the plants classification dataset.

In [ ]:
import json

# Plot the training and validation accuracy and loss for the model.
def plot_training_vs_validation(history):
    acc = history['accuracy']
    val_acc = history['val_accuracy']
    loss = history['loss']
    val_loss = history['val_loss']
    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b', label='Training accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()


with open(training_history_path, 'r') as f:
    history = json.load(f)
    plot_training_vs_validation(history)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import zipfile

# Extract the test images in ZIP format into a temporary directory.
test_dir = 'test-images'
with zipfile.ZipFile(test_images_path, 'r') as zip_ref:
    zip_ref.extractall(test_dir)

# Create a test file generator for the model to run predictions on.
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir + '/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Get the model's predictions for the test images.
predictions = model.predict(test_generator, steps=len(test_generator), verbose=1)
predicted_labels = np.argmax(predictions, axis=1)

def plot_confusion_matrix(true_labels, predicted_labels, class_labels):
    cm = confusion_matrix(true_labels, predicted_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)

    plt.figure(figsize=(14, 14))
    disp.plot(cmap=plt.cm.Blues, values_format='d')
    plt.title('Confusion Matrix')
    plt.xticks(rotation=90)
    plt.show()

plot_confusion_matrix(test_generator.classes, predicted_labels, plant_species)